In [59]:
import pandas as pd
import numpy as np

In [131]:
data = pd.read_csv('data/train_data.csv', error_bad_lines=False)

/var/folders/51/svyxncy90c19w1j2cby059bm0000gn/T/ipykernel_19821/2132394297.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('data/train_data.csv', error_bad_lines=False)
b'Skipping line 5856: expected 40 fields, saw 70\nSkipping line 7939: expected 40 fields, saw 49\n'


In [32]:
lot_f = 'LTFRONT'
lot_d = 'LTDEPTH'
bld_f = 'BLDFRONT'
bld_d = 'BLDDEPTH'

area_data = pd.DataFrame(data[(data[lot_f] > 0) & (data[lot_d] > 0) & (data[bld_f] > 0) & (data[bld_d] > 0)], columns=[lot_f, lot_d, bld_f, bld_d])

In [41]:
front_mean = (area_data[lot_f] - area_data[bld_f]).mean()
front_std = (area_data[lot_f] - area_data[bld_f]).std()
depth_mean = (area_data[lot_d] - area_data[bld_d]).mean()
depth_std = (area_data[lot_d] - area_data[bld_d]).std()


print(f'Front: mean {front_mean}\t std: {front_std}\nDepth: mean {depth_mean}\t std: {depth_std}')

Front: mean 11.060518034374244	 std: 45.07528268290128
Depth: mean 49.48995400629388	 std: 49.1858577008738


In [103]:
def get_mean_column_value_for_boro(data, column_name: str, boro: int, include_zeros: bool = False):
    column_vals = data[data['BORO'] == boro][column_name]
    if include_zeros:
        return np.mean(column_vals)
    else:
        return np.mean(column_vals[column_vals != 0])

In [134]:
def fill_missing_column_values_with_mean_for_boro(data, column_name: str):
    data_filled = data.copy()
    for i, row in data_filled.iterrows():
        data_filled.at[i, column_name] = get_mean_column_value_for_boro(data_filled, column_name, row['BORO'])
    return data_filled

In [130]:
def replace_lengths_with_areas(data):
    data_with_area = data.copy()
    lot_f = 'LTFRONT'
    lot_d = 'LTDEPTH'
    bld_f = 'BLDFRONT'
    bld_d = 'BLDDEPTH'
    data_with_area['LTAREA'] = data[lot_f] * data[lot_d]
    data_with_area['BLDAREA'] = data[bld_f] * data[bld_d]
    data_with_area['LTAREA'] = np.where(data_with_area['LTAREA'] == 0, data_with_area['BLDAREA'], data_with_area['LTAREA'])
    data_with_area['BLDAREA'] = np.where(data_with_area['BLDAREA'] == 0, data_with_area['LTAREA'], data_with_area['BLDAREA'])
    
    for len_col_name in [lot_f, lot_d, bld_f, bld_d]:
        del data_with_area[len_col_name]
    return data_with_area

In [57]:
data_with_area = data.copy()
data_with_area['LTAREA'] = data[lot_f] * data[lot_d]
data_with_area['BLDAREA'] = data[bld_f] * data[bld_d]

In [61]:
data_with_area['LTAREA'] = np.where(data_with_area['LTAREA'] == 0, data_with_area['BLDAREA'], data_with_area['LTAREA'])

In [132]:
data_with_area = replace_lengths_with_areas(data)

In [137]:
data_with_area_filled = fill_missing_column_values_with_mean_for_boro(data_with_area, 'LTAREA')
data_with_area_filled = fill_missing_column_values_with_mean_for_boro(data_with_area_filled, 'BLDAREA')

In [113]:
get_mean_column_value_for_boro(data_with_area, 'LTAREA', 1)

20278.494930875644

In [93]:
column_vals = data_with_area[data['BORO'] == 1]['LTAREA']